In [1]:
def skull_stripping():
    import vtkITK
    #masterVolumeNode = slicer.util.loadVolume('E:/asdgdrhss/8 CTA_1_0__H20f_5.nrrd')
    for _ in range (76):
        masterVolumeNode = slicer.mrmlScene.GetFirstNodeByClass("vtkMRMLScalarVolumeNode")
        threshold = 500
        split_cavities = 30

        thresholdCalculator = vtkITK.vtkITKImageThresholdCalculator()
        thresholdCalculator.SetInputData(masterVolumeNode.GetImageData())
        thresholdCalculator.SetMethodToOtsu()
        thresholdCalculator.Update()
        boneThresholdValue = thresholdCalculator.GetThreshold()
        volumeScalarRange = masterVolumeNode.GetImageData().GetScalarRange()
        logging.info("Volume minimum = {0}, maximum = {1}, bone threshold = {2}".format(volumeScalarRange[0], volumeScalarRange[1], boneThresholdValue))
        slicer.app.processEvents()

        # Create segmentation
        slicer.app.processEvents()
        segmentationNode = slicer.vtkMRMLSegmentationNode()
        slicer.mrmlScene.AddNode(segmentationNode)
        segmentationNode.CreateDefaultDisplayNodes() # only needed for display
        segmentationNode.SetReferenceImageGeometryParameterFromVolumeNode(masterVolumeNode)

        # Create segment editor to get access to effects
        slicer.app.processEvents()
        segmentEditorWidget = slicer.qMRMLSegmentEditorWidget()
        # To show segment editor widget (useful for debugging): segmentEditorWidget.show()
        segmentEditorWidget.setMRMLScene(slicer.mrmlScene)
        if not segmentEditorWidget.effectByName("Wrap Solidify"):
            slicer.util.errorDisplay("Please install 'SurfaceWrapSolidify' extension using Extension Manager.")

        segmentEditorNode = slicer.vtkMRMLSegmentEditorNode()
        slicer.mrmlScene.AddNode(segmentEditorNode)
        segmentEditorWidget.setMRMLSegmentEditorNode(segmentEditorNode)
        segmentEditorWidget.setSegmentationNode(segmentationNode)
        segmentEditorWidget.setMasterVolumeNode(masterVolumeNode)

        # Create bone segment by thresholding
        slicer.app.processEvents()
        boneSegmentID = segmentationNode.GetSegmentation().AddEmptySegment("bone")
        segmentEditorNode.SetSelectedSegmentID(boneSegmentID)
        segmentEditorWidget.setActiveEffectByName("Threshold")
        effect = segmentEditorWidget.activeEffect()
        effect.setParameter("MinimumThreshold",str(threshold))#change if needed
        effect.setParameter("MaximumThreshold",str(volumeScalarRange[1]))
        effect.self().onApply()

        # Find largest object, remove all other regions from the segment
        slicer.app.processEvents()
        segmentEditorWidget.setActiveEffectByName("Islands")
        effect = segmentEditorWidget.activeEffect()
        effect.setParameterDefault("Operation", "KEEP_LARGEST_ISLAND")
        effect.self().onApply()

        # Create Margin
    #     slicer.app.processEvents()
    #     segmentEditorWidget.setActiveEffectByName("Margin")
    #     effect = segmentEditorWidget.activeEffect()
    #     effect.setParameterDefault("Operation", "Grow")
    #     effect.setParameter('Margin_Size', 1)
    #     effect.self().onApply()



        # Fill holes in the segment to create a solid region of interest
        slicer.app.processEvents()
        segmentEditorWidget.setActiveEffectByName("Wrap Solidify")
        effect = segmentEditorWidget.activeEffect()
        effect.setParameter("region", "largestCavity")
        effect.setParameter("splitCavities", str(split_cavities))
        effect.setParameter("outputType", "segment")
        #effect.setParameter("smoothingFactor", 0.2)  # speed up solidification by lowering resolution
        #effect.setParameter("remeshOversampling", 1.5)  # speed up solidification by lowering resolution
        #effect.setParameter("numberOfIterations", 6)  # speed up solidification by lowering resolution
        effect.self().onApply()



        # Blank out the volume outside the object segment
        slicer.app.processEvents()
        segmentEditorWidget.setActiveEffectByName('Mask volume')
        effect = segmentEditorWidget.activeEffect()
        effect.setParameter('FillValue', -1000)
        effect.setParameter('Operation', 'FILL_OUTSIDE')
        effect.self().onApply()

        # Remove temporary nodes and widget
        segmentEditorWidget = None
        slicer.mrmlScene.RemoveNode(segmentEditorNode)
        slicer.mrmlScene.RemoveNode(segmentationNode)
        slicer.mrmlScene.RemoveNode(masterVolumeNode)

        # Show masked volume
        maskedVolume = slicer.mrmlScene.GetFirstNodeByName(masterVolumeNode.GetName()+" masked")
        slicer.util.setSliceViewerLayers(background=maskedVolume)
    
    
shortcuts = [("s", lambda: skull_stripping())]
for (shortcutKey, callback) in shortcuts:
  shortcut = qt.QShortcut(slicer.util.mainWindow())
  shortcut.setKey(qt.QKeySequence(shortcutKey))
  shortcut.connect( "activated()", callback)

Volume minimum = -1038.0, maximum = 3095.0, bone threshold = -392.117919921875
1 islands created (117 ignored)
Volume minimum = -1033.0, maximum = 2892.0, bone threshold = -419.6229248046875
1 islands created (216 ignored)
Volume minimum = -1036.0, maximum = 2906.0, bone threshold = -419.96624755859375
1 islands created (214 ignored)
Volume minimum = -1037.0, maximum = 2908.0, bone threshold = -420.4974365234375
1 islands created (190 ignored)
Volume minimum = -1040.0, maximum = 2887.0, bone threshold = -426.31036376953125
1 islands created (201 ignored)
Volume minimum = -1038.0, maximum = 3095.0, bone threshold = 835.05810546875
1 islands created (155 ignored)
Volume minimum = -1042.0, maximum = 3094.0, bone threshold = 832.41796875
1 islands created (202 ignored)
Volume minimum = -1036.0, maximum = 3085.0, bone threshold = 767.218994140625
1 islands created (138 ignored)
Volume minimum = -1038.0, maximum = 3087.0, bone threshold = 766.969482421875
1 islands created (85 ignored)
Volum

1 islands created (90 ignored)
Volume minimum = -1043.0, maximum = 3086.0, bone threshold = -397.742919921875
1 islands created (72 ignored)
Volume minimum = -1041.0, maximum = 3084.0, bone threshold = -396.3680419921875
2 islands created (92 ignored)
